读取SAM文件中每一条read的比对位置（包含染色体编号和位置信息），比对质量值和插入片段长度输出，可以这样做：

In [ ]:
import pysam
bf = pysam.AlignmentFile('ex10.sam', 'r')
for r in bf:
  print r.reference_name, r.pos, r.mapq, r.isize

读取SAM文件中特定区域的信息，比如与chr2第80-120位比对的结果，可以使用fetch函数：

In [ ]:
import pysam
bf = AlignmentFile('ex3.sam', 'r')
for r in bf.fetch('chr2', 80, 120):
  print r
bf.close()

当要求更高，比如说要输出覆盖了特定位置，且比对质量大于20的所有reads，可以使用Alignment模块的pileups函数来解决：

In [ ]:
import pysam
bf = pysam.AlignmentFile("ex3.sam", "r" )
for pileupcolumn in bf.pileup("chr2", 80, 81):
    for read in [al for al in pileupcolumn.pileups if al.alignment.mapq>20]:
        if not read.is_del and not read.is_refskip:
         	if read.alignment.pos + 1 == 81:
                print read.alignment.reference_name,\
                      read.alignment.pos + 1,\
                    read.alignment.query_sequence[read.query_position]

bf.close()

最后，关于如何使用pysam包在已有的完整信息的基础上生成sam文件，具体做法如下：

In [ ]:
rt pysam

header = {'HD': {'VN': '1.0'},
          'SQ': [{'LN': 1575, 'SN': 'chr1'},
                 {'LN': 1584, 'SN': 'chr2'}]
}
tmpfilename = "out.sam"
with pysam.AlignmentFile(tmpfilename, "wb", header=header) as outf:
    a = pysam.AlignedSegment()  # 定义一个AlignedSegment对象用于存储比对信息
    a.query_name = "read_28833_29006_6945"
    a.query_sequence="AGCTTAGCTAGCTACCTATATCTTGGTCTTGGCCG"
    a.flag = 99
    a.reference_id = 0
    a.reference_start = 32
    a.mapping_quality = 20
    a.cigar = ((0,10), (2,1), (0,25))
    a.next_reference_id = 0
    a.next_reference_start=199
    a.template_length=167
    a.query_qualities = pysam.qualitystring_to_array("<<<<<<<<<<<<<<<<<<<<<:<9/,&,22;;<<<")
    a.tags = (("NM", 1),
              ("RG", "L1"))
    outf.write(a)